In [1]:
from utility_dfcounter import *

## 1. count nominal

In [3]:
def countDataFrames(variation=""):
    labels  = ["trigger","usetag","acc","accVar","nmcbg","nmcbgVar","nfake","nfakeVar","ndata","ndataVar"]
    records = []
    
    for trigger in ["mu","e"]:
        for usetag in ["1b","2b"]:
            
            print( "counting "+trigger+usetag + " ...")

            counter = DFCounter(trigger,usetag)
            counter.setVariation(variation)

            acc,accVar = counter.returnAcc()
            nmcbg,nmcbgVar = counter.returnNMCbg()
            nfake,nfakeVar = counter.returnNFake()
            ndata,ndataVar = counter.returnNData()
            records.append( (trigger,usetag,acc,accVar,nmcbg,nmcbgVar,nfake,nfakeVar,ndata,ndataVar) )

    df = pd.DataFrame.from_records(records, columns=labels)
    df.to_pickle( common.getBaseDirectory() + "data/counts/count_{}.pkl".format(variation))
    print( "counting finished!")
    
# def countDataFrames_selection(variation=""):
#     labels  = ["selection","nbjet","acc","accVar","nmcbg","nmcbgVar","nfake","nfakeVar","ndata","ndataVar"]
#     records = []
    
#     for selection in ["mumu","ee"]:
#         for nbjet in ["==1",">1"]:

#             counter = DFCounter_selection(selection,nbjet)
#             counter.setVariation(variation)

#             acc,accVar = counter.returnAcc()
#             nmcbg,nmcbgVar = counter.returnNMCbg()
#             nfake,nfakeVar = np.zeros(2), np.zeros(2)
#             ndata,ndataVar = counter.returnNData()
#             records.append( (selection,nbjet,acc,accVar,nmcbg,nmcbgVar,nfake,nfakeVar,ndata,ndataVar) )

#     df = pd.DataFrame.from_records(records, columns=labels)
#     df.to_pickle( common.getBaseDirectory() + "data/counts/DNNcount_{}.pkl".format(variation))
#     print( "counting finished!")

In [4]:
#countDataFrames()

## 2. count with variation

In [5]:
import multiprocessing as mp

def runCountDataFrames(vlist):
    processes = []
    for v in vlist:
        processes.append(mp.Process(target=countDataFrames,args=(v,)))
    for ps in processes:
        ps.start()
    for ps in processes:
        ps.join()

In [6]:
# runCountDataFrames(['EPtDown','MuPtDown','TauPtDown',"TTXSUp","TWXSUp","",
#                     "JESUp","JESDown","JERUp","JERDown",
#                     "BTagUp","BTagDown","MistagUp","MistagDown"]) 

# runCountDataFrames(["RenormUp","RenormDown","FactorUp","FactorDown","PDFUp","PDFDown",
#                     'FSRUp','FSRDown','ISRUp','ISRDown','UEUp','UEDown','MEPSUp','MEPSDown']) 

# runCountDataFrames(['FSRUp','FSRDown','ISRUp','ISRDown']) # ttTheory
# runCountDataFrames(['UEUp','UEDown','MEPSUp','MEPSDown']) # ttTheory

#runCountDataFrames(["RenormUp","RenormDown","FactorUp","FactorDown"]) # ttLHEWeight
runCountDataFrames(["PDFUp","PDFDown","TTXSUp","TWXSUp"]) # ttLHEWeight
runCountDataFrames(['EPtDown','MuPtDown','TauPtDown',""]) # lepEnergy
runCountDataFrames(["JESUp","JESDown","JERUp","JERDown"]) # jetEnergy
runCountDataFrames(["BTagUp","BTagDown","MistagUp","MistagDown"]) # bTagging

counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting finished!
counting finished!
counting finished!
counting finished!
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting finished!
counting finished!
counting finished!
counting finished!
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting

## 3. Count uncertainty of Tau acc

In [6]:
# jet misID
df = DFCutter('etau','>1',"mctt").getDataFrame()
np.sum( df.eventWeight*(1+0.002*df.lepton2_pt) )/ np.sum(df.eventWeight)

1.0798576525660608

# 4. Print the Expect Yields Table

In [2]:
ct = DFCounter('e','1b')
selection = 'etau'

total,totalVar = 0,0


df = DFCutter(selection,ct.nbjet,"mctt").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mct").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mcw").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mcz").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mcdiboson").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

n, nVar = ct.getNFake(selection,ct.nbjet)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

print("{:8.1f} +/- {:4.1f}".format(total,totalVar**0.5) )
df = DFCutter(selection,ct.nbjet,"data2016").getDataFrame()
print("{:8.1f} +/- {:4.1f}".format(np.sum(df.eventWeight),np.sum(df.eventWeight**2)**0.5) )

 11703.2 +/- 45.4
   705.6 +/- 29.2
   184.4 +/- 20.8
   340.6 +/- 51.0
    15.2 +/-  1.2
   483.1 +/- 63.8
 13432.2 +/- 100.1
 13213.0 +/- 114.9


# 5. Summarize Counts

In [3]:
counts = summaryCounts()
counts.to_pickle( common.getBaseDirectory() + "data/counts/count_summary.pkl")

In [4]:
counts.ndata[0]

array([ 42450.,  70548.,  18718., 579330.,  13669.,  20246.,   5100.,
       188271.,  27349.,  49637.,  13213., 412463.,   8116.,  15497.,
         3534., 132564.])